# Calculating population density

We want to know how many individuals are exposed to flares at each of the study regions.

To do this we want to:
- read in all population data from the study regions
- calculate the total population density within each of the study regions
-  using literatiure select an appropriate threshhold for AOD to use as a cutoff point for the "high levels of pollution"
- from this calculate total no. of people and % of people in area affected by high levels of pollution

###  Literature
Taking a look at this articles:
- [What Can Affect AOD–PM2.5 Association?](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2854780/)
- [An empirical relationship between PM2.5 and aerosol optical depth in Delhi Metropolitan](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3237057/)
  - "1% change in AOD explains 0.52±0.202% and 0.39±0.15% change in PM2.5 monitored within ±45 and 150 min intervals of AOD data."
    - [Table with PM2.5 to AOD conversions (for Dehli](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3237057/table/T1/)
      - PM2.5 (μg m−3) 30.5 = AOD (5 km) 172.1 

In [ ]:
import os

# Reading in all population data
import geopandas as gpd
base_dir = os.path.dirname(os.getcwd())

In [ ]:
gwer_road = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_gwer_polygon.geojson")
kalak = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_kalak_polygon.geojson")
lalish = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_lalish_polygon.geojson")

In [ ]:
gwer_road.head()

In [ ]:
sum(gwer_road["VALUE"])

In [ ]:
sum(gwer_road["VALUE"])

In [ ]:
sum(kalak["VALUE"])

In [ ]:
sum(lalish["VALUE"])

Next we want to read in the `AOD` data which quanitifies atmospheric haziness and gives us an idea of the level of pollution in the atmosphere.


In [ ]:
gwer_road_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_gwer_polygon.geojson")
kalak_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_kalak_polygon.geojson")
lalish_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_lalish_polygon.geojson")

In order to understand the exposure to `AOD` we need to count the population exposed to `AOD` concentrations above an average of the following threshhold:
- (170.2/30.5 μg/m3 )*5 μg/m3  = 27 μg/m3  [source here](https://www.ersnet.org/wp-content/uploads/2021/10/WHO-AQG_Joint-Society-Statement_1st-UPDATE-13th-October.pdf)

In [ ]:
gwer_road_mean_aod = gwer_road_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

kalak_mean_aod = kalak_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

lalish_mean_aod = lalish_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

Calculate the percentage of population exposed to rates of PM2.5 (using the AOD-> PM2.5 conversion) exceeding the WHO annual mean concentrations recommendations.

In [ ]:
gwer_road_harmful_aod = gwer_road_mean_aod[gwer_road_mean_aod["Optical_Depth_047"]>175]
kalak_harmful_aod = kalak_mean_aod[kalak_mean_aod["Optical_Depth_047"]>175]
lalish_harmful_aod = lalish_mean_aod[lalish_mean_aod ["Optical_Depth_047"]>175]

In [ ]:
sum(gwer_road_harmful_aod["VALUE"])/sum(gwer_road_mean_aod['VALUE'])

In [ ]:
sum(kalak_harmful_aod["VALUE"])/sum(kalak_mean_aod['VALUE'])

In [ ]:
sum(lalish_harmful_aod["VALUE"])/sum(lalish_mean_aod['VALUE'])

Exposure rates as follow:
- 100% of kalak population above WHO recommended levels of annual mean concentrations of PM2.5
- 100% of gwer road population above WHO recommended levels of annual mean concentrations of PM2.5
- 99.5% of lalish population above WHO recommended levels of annual mean concentrations of PM2.5

Calculating overall average to get figure for no. of times greater than the WHO recommended limit is for each study region, assuming that `(170.2 AOD / 30.5 μg/m3 ) * 5 μg/m3 (WHO limit) = 27 AOD`

In [ ]:
gwer_road_aod["Optical_Depth_047"].mean()/27

In [ ]:
kalak_aod["Optical_Depth_047"].mean()/27

In [ ]:
lalish_aod["Optical_Depth_047"].mean()/27

Calculating overall average to get figure for no. of times greater than the WHO recommended limit is for each study region, assuming that `(170.2 AOD / 30.5 μg/m3 ) * 5 μg/m3 (WHO limit) = 27 AOD`
- gwer_road = 7.15 x WHO Limit
- Kalak = 7.16 x WHO Limit
- lalish = 7.56 x WHO Limit